# Memory Management

# Variables are Memory References

In [2]:

my_var = 10

id( my_var )

140703527811136

In [3]:
hex( id ( my_var ))

'0x7ff817cb3840'

In [4]:
greeting = "hello"

hex( id ( greetingeting))

'0x1e81eb14170'

### Observe: both have different address

## Reference Counting

In [5]:
import ctypes

In [6]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value


In [7]:
my_var = [1, 2, 3, 4]
ref_count(id(my_var))

1

## one more method

In [8]:
import sys
sys.getrefcount(my_var)

2

In [16]:
other_var = my_var

In [17]:
print(hex(id(my_var)), hex(id(other_var)))

print(ref_count(id(my_var)))

0x1e81dd84880 0x1e81dd84880
2


In [18]:
other_var = None

In [19]:
print(ref_count(id(my_var)))

1


### We see that the reference count has gone back to 1.

#### In Python-- Memory management is completely transparent -

# Garbage Collection

In [20]:
import ctypes
import gc

In [21]:
def object_by_id(object_id):
    ''' search for object in the GC w r t id '''
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"


#### define two classes that we will use to create a circular reference

In [22]:
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b:{1}'.format(hex(id(self)), hex(id(self.b))))

In [23]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))
        

We turn off the GC so we can see how reference counts are affected when the GC does not run and when it does (by running it manually).

In [24]:
gc.disable()

In [25]:
my_var = A()

B: self: 0x1e81ce20340, a: 0x1e81ce20a90
A: self: 0x1e81ce20a90, b:0x1e81ce20340


u can see a circular reference.<br>
In fact `my_var` is also a reference to the same A instance:

In [26]:
print(hex(id(my_var)))

0x1e81ce20a90


In [27]:
print('a: \t{0}'.format(hex(id(my_var))))
print('a.b: \t{0}'.format(hex(id(my_var.b))))
print('b.a: \t{0}'.format(hex(id(my_var.b.a))))


a: 	0x1e81ce20a90
a.b: 	0x1e81ce20340
b.a: 	0x1e81ce20a90


In [28]:
a_id = id(my_var)
b_id = id(my_var.b)


In [29]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 2
refcount(b) = 1
a: Object exists
b: Object exists


In [30]:
my_var= None

In [31]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 1
refcount(b) = 1
a: Object exists
b: Object exists


As we can see, the reference counts are now both equal to 1 (a pure circular reference), and reference counting alone did not destroy the A and B instances - they're still around. If no garbage collection is performed this would result in a memory leak.

Let's run the GC manually and re-check whether the objects still exist:

In [33]:
gc.collect()
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 0
refcount(b) = 0
a: Not found
b: Not found


GC removed memory leak

# Dynamic Typing and Static Typing

Some languages (Java, C++, Swift) are statically typed.<br>

String  myVar = “hello”;<br>
data  variable   value
type  name




Python, in contrast, is dynamically typed.

In [40]:
my_var = 'hello'

print( type( my_var) )
id(my_var)

<class 'str'>


2096458973552

In [38]:
my_var = 10

id(my_var)

140703527811136

In [39]:
type( my_var )

int

Observe: **type** is changed more often.

### variable re-assignment

Notice how the memory address of **a** is different every time.

In [45]:
a = 10
hex(id(a))


'0x7ff817cb3840'

In [46]:
a = 15
hex(id(a))


'0x7ff817cb38e0'

In [47]:
a = 5
hex(id(a))


'0x7ff817cb37a0'

In [48]:
a = a + 1
hex(id(a))


'0x7ff817cb37c0'

### Look at this

In [49]:
a = 10
b = 10
print(hex(id(a)))
print(hex(id(b)))


0x7ff817cb3840
0x7ff817cb3840


The memory addresses of both **a** and **b** are the same!! 

# Object Mutability

Immutable Mutable:

• Numbers (int, float, Booleans, etc)<br>
• Strings<br>
• Tuples<br>
• Frozen Sets<br>
• User-Defined Classes<br>
• Lists<br>
• Sets<br>
• Dictionaries<br>
• User-Defined Classes<br>

Mutable<br>
• Lists<br>
• Sets<br>
• Dictionaries<br>
• User-Defined Classes<br>

The object references in the tuple did not change
but the referenced objects did mutate!

In [50]:
t = (1, 2, 3) # tuple is imutable: these are references to immutable object (int)

In [51]:
t = ([1, 2], [3, 4]) # tuple is immutable: these are references to a mutable object (list)

# Function arguments and Mutability

In [52]:

def process(s):
    s = s + 'world'
    return s


In [53]:
my_var = 'hello'
process( my_var )

'helloworld'

In [54]:
print(my_var)

hello


strings are immutable objects.<br>
#### Immutable objects are safe from unintended side-effects

### Mutable objects are not safe from unintended side-effects

In [55]:

def process( lst ): #my_lists’s reference is passed to process()
    lst.append( 100 )
    
my_list = [ 1,2,3 ]


process( my_list )

print( my_list )

[1, 2, 3, 100]


#### Immutable collection objects that contain mutable objects

In [56]:

def process(t):
    t[0].append(3)
    
    
my_tuple = ([1,2], 'a' )

process(my_tuple)

print(my_tuple)


([1, 2, 3], 'a')


# Shared references and Mutability

The term shared reference is the concept of two variables referencing
the same object in memory (i.e. having the same memory address)

In [57]:
a = 10
b = a


In [58]:
print(id ( a ))
print( id( b ))

140703527811136
140703527811136


In [59]:
s1 = "hello"
s2 = "hello"

print(id ( s1 ))
print( id( s2 ))

2096458973552
2096458973552


In both these cases, Python’s memory manager decides to automatically re-use
the memory references!!

Is this even safe? Yes

The integer object 10, and the string object ‘hello’ are immutable - so it is safe to set up a shared reference

#### When working with mutable objects we have to be more careful

In [61]:
a = [ 1,2,3]
b = a


In [62]:
b.append( 100 )

In [63]:
print(id(a))

2096444774464


In [64]:
id(b)

2096444774464

In [65]:
print(a) #

[1, 2, 3, 100]


In [66]:
b

[1, 2, 3, 100]

Both a and b also changed

#### With mutable objects, the Python memory manager will never create shared references

# Variable Equality

is -> ( identity operator) memory address.<br>
== -> ( equality operator ) Object data.<br>

is not <br>
!=

In [67]:
a = 10
b = a

In [68]:
a is b

True

In [69]:
a == b

True

In [70]:
a = [ 1,2,3]
b = [ 1,2,3]

In [72]:
a is b # due to mutable

False

In [73]:
a == b

True

In [74]:
a = 10
b = 10.0

In [76]:
a is b # due to data type

False

In [77]:
a == b

True

The `None` object<br>
The None object can be assigned to variables to indicate that they are not set.i.e. an “empty” value (or null pointer).<br>

But the None object is a real object that is managed by the Python memory manager.<br>

the memory manager will always use a shared reference when assigning
a variable to None.<br>



In [78]:
a = None
b = None
c = None


print(id(a), id(b))

140703527577728 140703527577728


In [79]:
a is None

True

In [80]:
x = 10

x is None

False

In [81]:
x is not None

True

# Every thing is an Object

Till now: <br>
• Integers (int)<br>
• Booleans (bool)<br>
• Floats (float)<br>
• Strings (str)<br>
• Lists (list)<br>
• Tuples (tuple)<br>
• Sets (set)<br>
• Dictionaries (dict)<br>
• None (NoneType)<br>


• Operators (+, -, ==, is, …)<br>
• Functions<br>
• Classes<br>
• Types<br>


In [83]:

def fun():
    pass

id( fun() )

140703527577728

In [84]:
class A:
    pass


id(A)

2096417840240

In [85]:
type( type )

type

But the one thing in common with all these things, is that
they are all objects (instances of classes).<br>

This means they all have a memory address!<br>

Any object can be assigned to a variable.<br>
Any object can be passed to a function.<br>
Any object can be returned from a function